In [1]:
import pandas as pd 
from pytrends.request import TrendReq
from datetime import datetime 

In [2]:
# hl specifies host language for accessing Google Trends, tz is Timezone offset, 360 is for US CST.
pytrends = TrendReq(hl='en-US', tz=360, timeout=(5, 7))

In [3]:
kw = "Bitcoin"

In [4]:
df = pytrends.get_historical_interest([kw], year_start=2022, month_start=1, day_start=1, year_end=2022, month_end=9, day_end=15, geo='', frequency="daily")



/Users/hongyuan/personal_project/dev/google_trends/google_trends_venv/lib/python3.8/site-packages/pytrends/request.py:589: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(week_df)
/Users/hongyuan/personal_project/dev/google_trends/google_trends_venv/lib/python3.8/site-packages/pytrends/request.py:611: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(week_df)


In [5]:
df.head(30)

,Bitcoin,isPartial
date,,
2022-01-01,36,False
2022-01-02,36,False
2022-01-03,37,False
2022-01-04,45,False
2022-01-05,35,False
2022-01-06,53,False
2022-01-07,61,False
2022-01-08,54,False
2022-01-09,50,False


In [6]:
geo = {
    "China": "CN",
    "Canada": "CA",
    "United States": "US",
    "Global": ''
}

In [7]:
def collect_bitcoin_trends(search_range='', start_time="2015-01-01", kw_list=["Bitcoin"]):
    now = datetime.now()
    start_y, start_m, start_d = [int(item) for item in start_time.split("-")]
    pytrends = TrendReq(hl='en-US', tz=360, timeout=(5, 7))
    df = pytrends.get_historical_interest(kw_list, year_start=start_y, month_start=start_m, day_start=start_d, year_end=now.year, month_end=now.month, day_end=now.day, geo=search_range, frequency="daily")
    return df
    

In [8]:
df = collect_bitcoin_trends()

/Users/hongyuan/personal_project/dev/google_trends/google_trends_venv/lib/python3.8/site-packages/pytrends/request.py:589: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(week_df)
/Users/hongyuan/personal_project/dev/google_trends/google_trends_venv/lib/python3.8/site-packages/pytrends/request.py:589: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(week_df)
/Users/hongyuan/personal_project/dev/google_trends/google_trends_venv/lib/python3.8/site-packages/pytrends/request.py:589: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(week_df)
/Users/hongyuan/personal_project/dev/google_trends/google_trends_venv/lib/python3.8/site-packages/pytrends/request.py:589: FutureWarning: The frame.append

In [9]:
df

,Bitcoin,isPartial
date,,
2015-01-01,51,False
2015-01-02,51,False
2015-01-03,57,False
2015-01-04,69,False
2015-01-05,67,False
...,...,...
2022-09-08,58,False
2022-09-09,66,False
2022-09-10,54,False


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2823 entries, 2015-01-01 to 2022-09-12
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   Bitcoin    2823 non-null   int64
 1   isPartial  2823 non-null   bool 
dtypes: bool(1), int64(1)
memory usage: 46.9 KB


In [11]:
# df.to_csv('example.csv') 

In [12]:
# remove the rows with duplicated index
new_df = df[~df.index.duplicated(keep='first')]

In [13]:
new_df

,Bitcoin,isPartial
date,,
2015-01-01,51,False
2015-01-02,51,False
2015-01-03,57,False
2015-01-04,69,False
2015-01-05,67,False
...,...,...
2022-09-08,58,False
2022-09-09,66,False
2022-09-10,54,False


In [14]:
new_df.to_csv('bitcoin_daily_trends_data.csv') 

In [15]:
len(new_df.index)

2812

In [16]:
def generate_bitcoin_weekly_trends_data(daily_data):
    daily_data_2 = daily_data.loc[:, ['Bitcoin']]
    daily_data_2['Date'] = pd.to_datetime(daily_data.index, format="%Y%m%d")
    logic = {'Bitcoin': 'mean',
             'Date': 'first',
             }
    weekly_data = daily_data_2.groupby([daily_data_2['Date'].dt.isocalendar().year, daily_data_2['Date'].dt.isocalendar().week]).agg(logic)
    weekly_data.set_index('Date', inplace=True)
    weekly_data['Bitcoin'] = weekly_data['Bitcoin'].map(lambda x: '{0:.2f}'.format(x))
    weekly_data.to_csv('bitcoin_weekly_trends_data.csv') 
    

In [17]:
generate_bitcoin_weekly_trends_data(new_df)

In [18]:
new_df_2 = new_df.loc[:, ['Bitcoin']]

In [19]:
new_df_2

,Bitcoin
date,
2015-01-01,51
2015-01-02,51
2015-01-03,57
2015-01-04,69
2015-01-05,67
...,...
2022-09-08,58
2022-09-09,66
2022-09-10,54


In [20]:
new_df_2['Date'] = pd.to_datetime(new_df.index, format="%Y%m%d")
new_df_2

,Bitcoin,Date
date,,
2015-01-01,51,2015-01-01
2015-01-02,51,2015-01-02
2015-01-03,57,2015-01-03
2015-01-04,69,2015-01-04
2015-01-05,67,2015-01-05
...,...,...
2022-09-08,58,2022-09-08
2022-09-09,66,2022-09-09
2022-09-10,54,2022-09-10


In [21]:
logic = {
    'Bitcoin': 'mean',
    'Date': 'first',
}

In [22]:
weekly_data = new_df_2.groupby([new_df_2['Date'].dt.isocalendar().year, new_df_2['Date'].dt.isocalendar().week]).agg(logic)
weekly_data

Bitcoin       Date
year week                      
2015 1     57.000000 2015-01-01
     2     59.714286 2015-01-05
     3     68.428571 2015-01-12
     4     54.000000 2015-01-19
     5     58.714286 2015-01-26
...              ...        ...
2022 33    62.857143 2022-08-15
     34    59.714286 2022-08-22
     35    58.428571 2022-08-29
     36    59.428571 2022-09-05
     37    61.000000 2022-09-12

[403 rows x 2 columns]

In [23]:
weekly_data.set_index('Date', inplace=True)
weekly_data

,Bitcoin
Date,
2015-01-01,57.000000
2015-01-05,59.714286
2015-01-12,68.428571
2015-01-19,54.000000
2015-01-26,58.714286
...,...
2022-08-15,62.857143
2022-08-22,59.714286
2022-08-29,58.428571


In [24]:
weekly_data['Bitcoin'] = weekly_data['Bitcoin'].map(lambda x: '{0:.2f}'.format(x))
weekly_data

,Bitcoin
Date,
2015-01-01,57.00
2015-01-05,59.71
2015-01-12,68.43
2015-01-19,54.00
2015-01-26,58.71
...,...
2022-08-15,62.86
2022-08-22,59.71
2022-08-29,58.43
